# **Course Project 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.


# Download data

In [1]:
!gdown --id '1iMnGEit5-Z_kgQia8pZ5FIlC0LoW7fki' --output covid.train.csv
!gdown --id '1NOoy9gE1VnnbHluzaOoXG__PefYcvqXW' --output covid.test.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1iMnGEit5-Z_kgQia8pZ5FIlC0LoW7fki
To: /content/covid.train.csv
100% 2.00M/2.00M [00:00<00:00, 170MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1NOoy9gE1VnnbHluzaOoXG__PefYcvqXW
To: /content/covid.test.csv
100% 493k/493k [00:00<00:00, 174MB/s]


# Import packages

In [2]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

* same_seed function: fix random number for reproducing the training result.
    * **Input**: seed (a constant)
    * **Output**: none

* train_valid_split: split training data further into training_data + validation_data
    * **Input**: data_set (training dataset); valid_ratio (a constant less than 1); seed (a constant)
    * **Output**: train_set (training dataset); valid_set (validation dataset)

* predict: make model prediction after the model is completed for the training
    * **Input**: test_loader (testing data); model (deep neural network after the training); device (either GPU or CPU, the device where the model is running upon)
    * **Output**: preds (a list of numerical variables, the length is identical to the testing dataset).



In [3]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    ''' Make predictions after the model is completed from the training'''
    # Set your model to evaluation mode, which will disable dropout and batch normalization.
    # which is important for getting an accurate evaluation of the model's performance
    model.eval()
    preds = []  # a list for storing the prediction results
    for x in tqdm(test_loader):
        x = x.to(device)
        # no_grad disables the gradient calculation to reduce memory consumption
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    # torch.cat: concatenates tensors to form a column
    # .numpy(): convert tensors into normal numbers
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset

* A class for getting the dataset elements

In [4]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if y=none, make prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            # convert the numerical y into Tensor to accelerate the training
            self.y = torch.FloatTensor(y)
        # convert the numerical x into Tensor to accelerate the training
        self.x = torch.FloatTensor(x)

    # return x, y values if you know the index
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    # return the length of input x
    def __len__(self):
        return len(self.x)

# Neural Network Model
* You can try out different model architectures by modifying the class below.
* How to create nueral networks using Pytorch? [link](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

In [5]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
* Choose features that you think are useful.
    * Here, train_data is a 2-D list:
        * row -> training data peceies; column -> features.
    * label `y` is the last column.
    * features `x` is in all of the previous columns.
    * `:` in index indicates selecting all elements

In [9]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    # y (i.e., labels) are in the last column
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    # x (i.e., featuers) are in all previous columns
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

* Loss functions in Pytorch
    * nn.MSELoss (mean square error)
    * nn.L1Loss (mean absolute error)

* Optimization in Pytorch
    * torch.optim.SGD [link](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)

In [6]:
def trainer(train_loader, valid_loader, model, config, device):

    # Define your loss function, for regression, we use MSE
    criterion = nn.MSELoss(reduction='mean')

    # Define the optimization algorithm, here, we use the schocastic gradient decrease (SGD)
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)
    # For more optimizers, please reference https://pytorch.org/docs/stable/optim.html

    # Create directory of saving models.
    if not os.path.isdir('./models'):
        os.mkdir('./models')

    # Initialize training parameters
        # n_epochs: the number of training epochs (1 epoch sees all training data)
        # best_loss: the smallest loss so far
        # step: record the training steps
        # early_stop_count: count the times of early stop
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        # train_loader is the iterable training data, which contains x and y
        # The position parameter in tqdm is used to specify the position of the progress bar.
        # for the use of tdqm, check:
        # https://www.kaggle.com/code/nikhilkhetan/tqdm-tutorial#4.-Customizing-progress-bar-using-tqdm-parameters
        train_pbar = tqdm(train_loader, position=0)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your training data to GPU.
            pred = model(x)                     # Compute the output from the model given x
            loss = criterion(pred, y)           # Compute the loss value
            loss.backward()                     # Compute gradient of dL/dtheta (backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1                           # record the training step
            loss_record.append(loss.detach().item())    # record the loss values

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        # calculate the average value in training loss
        mean_train_loss = sum(loss_record)/len(loss_record)

        # Set your model to evaluation mode.
        model.eval()
        # empty the loss_record list
        loss_record = []
        # evaluate the model with the validation data
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        # calculate the average value in validation loss
        mean_valid_loss = sum(loss_record)/len(loss_record)
        # print the training and validation loss values
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

        # if the test results on validation data is better than the history, save the model
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            # reset the early_stop_count
            early_stop_count = 0
        else:
            # increment early_stop_count if mean_valid_loss is not improved
            early_stop_count += 1

        # if the number of early stop is larger than a pre-define value, print that the model is not improving
        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 3000,     # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [10]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 1728 x 118 (118 = id + 37 states + 16 features x 5 days)
# test_data size: 539 x 117 (without last day's positive numbers)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
# further split the train_data into training and valid datasets
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])



# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (1728, 118)
valid_data size: (432, 118)
test_data size: (539, 117)
number of features: 117


# Start training!

In [ ]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|██████████| 7/7 [00:03<00:00,  2.24it/s, loss=47]


Epoch [1/3000]: Train loss: 139.8268, Valid loss: 136.4604
Saving model with loss 136.460...


Epoch [2/3000]: 100%|██████████| 7/7 [00:00<00:00, 129.68it/s, loss=63.5]


Epoch [2/3000]: Train loss: 77.0922, Valid loss: 49.6951
Saving model with loss 49.695...


Epoch [3/3000]: 100%|██████████| 7/7 [00:00<00:00, 127.26it/s, loss=45.9]


...

Epoch [933/3000]: 100%|██████████| 7/7 [00:00<00:00, 116.07it/s, loss=2.02]


Epoch [933/3000]: Train loss: 2.1028, Valid loss: 2.1500


Epoch [934/3000]: 100%|██████████| 7/7 [00:00<00:00, 129.32it/s, loss=2.42]


Epoch [934/3000]: Train loss: 2.1765, Valid loss: 2.5787


Epoch [935/3000]:   0%|          | 0/7 [00:00<?, ?it/s, loss=2.27]

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['tested_positive'])
        writer.writerows(preds)

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

* Compare the prediction with true values


In [ ]:
!gdown --id '1c0HgE-5NhDjYIMiaNXMKrwVHU0R5zJXa' --output test_result.csv

In [ ]:
test_result = pd.read_csv('./test_result.csv').values

loss = preds - test_result
square_loss = np.square(loss)
mse = np.mean(square_loss)

print(mse)